In [2]:
# Step 1: Import core libraries
import pandas as pd
import numpy as np

# Step 2: Load the CSV data
df = pd.read_csv('../data/raw/2019-Nov.csv')

# Step 3: Show the top rows
df.head()


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
1,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2,2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,NaN,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
3,2019-11-01 00:00:01 UTC,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
4,2019-11-01 00:00:01 UTC,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2


In [3]:
# Check the structure of the data
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67501979 entries, 0 to 67501978
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 4.5+ GB


In [4]:
# Convert event_time to datetime format
df['event_time'] = pd.to_datetime(df['event_time'])


In [5]:
# Check how many nulls exist in each column
df.isnull().sum()


event_time              0
event_type              0
product_id              0
category_id             0
category_code    21898171
brand             9224078
price                   0
user_id                 0
user_session           10
dtype: int64

In [6]:
# Drop rows where user_id or product_id is missing
df = df.dropna(subset=['user_id', 'product_id'])

# Optional: Fill brand/category_code with 'unknown' if desired
df['brand'] = df['brand'].fillna('unknown')
df['category_code'] = df['category_code'].fillna('unknown')

In [7]:
# How many of each type of user action?
df['event_type'].value_counts()


event_type
view        63556110
cart         3028930
purchase      916939
Name: count, dtype: int64

In [8]:
# Sort events by time (just to be safe)
df = df.sort_values(by=['user_id', 'user_session', 'event_time'])


In [9]:
# Extract all purchase timestamps per session
purchase_df = df[df['event_type'] == 'purchase'][['user_session', 'event_time']]
purchase_df = purchase_df.rename(columns={'event_time': 'purchase_time'})


In [10]:
# Merge purchase time into original dataset (so each event knows if purchase happened)
df = pd.merge(df, purchase_df, on='user_session', how='left')


In [11]:
# Calculate how long (in seconds) after the event the purchase happened
df['time_to_purchase'] = (df['purchase_time'] - df['event_time']).dt.total_seconds()


In [14]:
# Step 1: Safely copy the cart event rows
cart_df = df[df['event_type'] == 'cart'].copy()

# Step 2: Define churn label (safe to modify now)
cart_df['churned'] = np.where(
    (cart_df['purchase_time'].isna()) | (cart_df['time_to_purchase'] > 604800),
    1,
    0
)



In [15]:
# For each session, count how many 'view' events occurred before the 'cart'
views_df = df[df['event_type'] == 'view']

# Count views per session
view_counts = views_df.groupby('user_session').size().reset_index(name='view_count')

# Merge with cart_df
cart_df = cart_df.merge(view_counts, on='user_session', how='left')
cart_df['view_count'] = cart_df['view_count'].fillna(0)  # Fill NaN if no views found


In [16]:
# First view time per session
first_view = views_df.groupby('user_session')['event_time'].min().reset_index()
first_view = first_view.rename(columns={'event_time': 'first_view_time'})

# Merge into cart_df
cart_df = cart_df.merge(first_view, on='user_session', how='left')

# Calculate time from first view to cart
cart_df['time_to_cart'] = (cart_df['event_time'] - cart_df['first_view_time']).dt.total_seconds()


In [18]:
# Just use the price column already in cart_df
cart_df['price'] = cart_df['price'].fillna(0)


In [19]:
# Day of week (0=Monday, 6=Sunday)
cart_df['cart_dayofweek'] = cart_df['event_time'].dt.dayofweek

# Hour of day (0–23)
cart_df['cart_hour'] = cart_df['event_time'].dt.hour


In [20]:
# Save to data/processed
cart_df.to_csv('../data/processed/cart_churn_dataset.csv', index=False)
